![msg](msg.jpg)

In [ ]:
#Imports 
import astropy.units as u
import astropy.constants as const
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 

from tqdm import tqdm

from DiscEvolution.grid import Grid
from DiscEvolution.star import SimpleStar
from DiscEvolution.eos import LocallyIsothermalEOS  
from DiscEvolution.disc import AccretionDisc
from DiscEvolution.viscous_evolution import ViscousEvolutionFV
from DiscEvolution.driver import PlanetDiscDriver

In [506]:
# Physical parameters
#Assumed
Mstar = 1.0 * u.solMass #Default 1 solmass 
Rstar = 1.0 * u.solRad #Default 1 solRad
mu = 2.33  # Mean molecular weight, assumed

#Given
M_disk_0 = 0.05 * u.solMass  #Initial Disk Mass, default is 0.05 solmass
gamma = 1.0
Rc = 1.0 * u.AU #Default 1AU
#Mdot_0 = 
t_final = 1 * u.Myr 

# LBP surface density profile with gamma = 1: Sigma(r) = (M_disk / (2*pi*Rc*r)) * exp(-r/Rc)
def Sigma_LBP_1(r) -> float:
    val = (M_disk_0)/(2* np.pi * Rc * r) * np.exp((-1*r)/(Rc))
    if (val.cgs.unit != (u.g / (u.cm)**2)):
        print("Sigma_1 error")
        return 0
    return val.cgs

def Temperature(r) -> float:
    T_0 = 150 * u.K 
    val  = T_0 * (r / (1.0 * u.AU))**(-1/2)
    if val.cgs.unit != u.K:
        print('Temperature error')
        return 0
    return val.cgs

#Taken from A2
def Omega(r) -> float:
    val = np.sqrt((const.G*Mstar)/(r**3))
    if (val.cgs.unit != (1 / u.s)): 
        print("Omega error")
        return 0
    return val.cgs

#Taken from A2
def c_s(r) -> float:
    val = np.sqrt((const.k_B*Temperature(r))/(mu*const.m_p))
    if (val.cgs.unit != (u.cm / u.s)): 
        print("c_s error")
        return 0
    return val.cgs

def H(r) -> float: 
    val = c_s(r)/Omega(r)
    if val.cgs.unit != (u.cm):
        print("H error")
        return 0
    return val.cgs

In [507]:
#Given Mdisk, Mdot(0), Rc -> Calculate alpha and Sigma0
M_disk_0 = 0.05 * u.solMass
Mdot_0 = 1E-7 * u.solMass / u.yr  #Armitage p.21
Rc = 1.0 * u.AU

#Armitage eq 74
#nu*Sigma = Mdot/(3pi) * (1 - sqrt(r_star/r))
##alpha * c_s * H * Sigma = (Mdot / 3*pi) * (1 - sqrt(r_star/r))
###alpha = (Mdot / 3*pi) / (c_s * H * Sigma) * (1 - sqrt(r_star/r))

def alpha(r) -> float:
    alpha = (Mdot_0 / (3*np.pi)) * (1/(c_s(r) * H(r) * Sigma_LBP_1(r))) * (1 - np.sqrt(Rstar/r))
    return alpha.cgs

r_ref = Rc #Not Rstar
alpha_val = alpha(r_ref)

Sigma_0 = Sigma_LBP_1(r_ref)

print(f"Accretion rate: {Mdot_0:.3e}")
print(f"At radius: {r_ref.to(u.AU):.3f}")
print(f"\nCalculated alpha: {alpha_val:.3e}")
print(f"Sigma_0: {Sigma_0.cgs:.3e}")


Accretion rate: 1.000e-07 solMass / yr
At radius: 1.000 AU

Calculated alpha: 8.973e-04
Sigma_0: 2.601e+04 g / cm2


In [508]:
# 1) Create grid
#R_in = Rstar.to('AU').value  # AU
R_in = 0.1 #AU
R_out = 30.0  # AU
Ncells = 200

grid = Grid(R_in, R_out, Ncells, spacing='log')

# 2) Create star
star = SimpleStar(M=1.0)  # Already in solar masses and radii

In [509]:
#1. turbulence dominated disk with fixed LBP T profile 

# 3) Create EOS - use a placeholder alpha, we'll override it
eos = LocallyIsothermalEOS(star, h0=H(Rc).value, q=-0.25, alpha_t=1.0)  # Placeholder
eos.set_grid(grid)

# 4) Create surface density array from LBP profile
R_centers = grid.Rc  # AU
Sigma_initial = np.array([Sigma_LBP_1(r * u.AU).value for r in R_centers])

# 5) Create the disc
disc = AccretionDisc(grid, star, eos, Sigma=Sigma_initial)
disc._planetesimal = False

# Override EOS with custom LBP temperature profile AND spatially varying alpha
T_custom = np.array([Temperature(r * u.AU).value for r in disc.R])
cs_custom = np.array([c_s(r * u.AU).value for r in disc.R])
H_custom = np.array([H(r * u.AU).value for r in disc.R])
alpha_custom = np.array([alpha(r * u.AU).value for r in disc.R])  # ← Alpha varies with r!

disc._eos._T = T_custom
disc._eos._cs = cs_custom
disc._eos._H = H_custom
disc._eos._nu = alpha_custom * cs_custom * H_custom  # ← Use spatially varying alpha
disc._eos._alpha_t = alpha_val.value  # Set scalar for reference

# Prevent disc.T property from recalculating incorrectly
disc._eos._T0 = 1.0

print(f"Custom temperature profile applied")
print(f"Temperature at 1 AU: {T_custom[np.argmin(np.abs(disc.R - 1.0))]:.1f} K")

# Verify setup
print(f"Grid: {disc.Ncells} cells from {R_in} to {R_out} AU")
print(f"Star mass: {disc._star.M:.3e}")
print(f"Disc mass: {disc.Mtot()/const.M_sun.to(u.g).value:.3e} solar masses")
print(f"Alpha at 1 AU: {alpha_custom[np.argmin(np.abs(disc.R - 1.0))]:.3e}")
print(f"Nu at 1 AU: {disc.nu[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm²/s")

Custom temperature profile applied
Temperature at 1 AU: 150.5 K
Grid: 200 cells from 0.1 to 30.0 AU
Star mass: 1.000e+00
Disc mass: 4.525e-02 solar masses
Alpha at 1 AU: 8.910e-04
Nu at 1 AU: 2.36e+13 cm²/s


In [510]:
# Diagnostic: Check if overrides are working
print(f"\n=== DIAGNOSTIC ===")
print(f"Custom H at 1 AU: {H_custom[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm")
print(f"disc.H at 1 AU: {disc.H[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm")
print(f"disc._eos._H at 1 AU: {disc._eos._H[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm")

print(f"\nCustom nu at 1 AU: {(disc._eos._alpha_t * cs_custom * H_custom)[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm²/s")
print(f"disc.nu at 1 AU: {disc.nu[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm²/s")
print(f"disc._eos._nu at 1 AU: {disc._eos._nu[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm²/s")

print(f"\nViscosity min: {disc.nu.min():.2e} cm²/s")
print(f"Viscosity max: {disc.nu.max():.2e} cm²/s")
print(f"Viscosity at 30 AU: {disc.nu[-1]:.2e} cm²/s")

# Check if the issue is that EOS is recalculating
print(f"\nEOS alpha_t: {disc._eos._alpha_t}")
print(f"Are disc.nu and disc._eos._nu the same object? {disc.nu is disc._eos._nu}")


=== DIAGNOSTIC ===
Custom H at 1 AU: 3.63e+11 cm
disc.H at 1 AU: 3.63e+11 cm
disc._eos._H at 1 AU: 3.63e+11 cm

Custom nu at 1 AU: 2.38e+13 cm²/s
disc.nu at 1 AU: 2.36e+13 cm²/s
disc._eos._nu at 1 AU: 2.36e+13 cm²/s

Viscosity min: 8.34e+11 cm²/s
Viscosity max: 1.93e+27 cm²/s
Viscosity at 30 AU: 1.93e+27 cm²/s

EOS alpha_t: 0.0008973302021235719
Are disc.nu and disc._eos._nu the same object? False
